In [2]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 25.0 MB/s eta 0:00:00


In [5]:
# Convert to Torch tensor (Ensure float64 for compatibility with Pennylane)
X_train_torch = torch.tensor(X_train, dtype=torch.float64)
y_train_torch = torch.tensor(y_train, dtype=torch.float64)
X_test_torch = torch.tensor(X_test, dtype=torch.float64)
y_test_torch = torch.tensor(y_test, dtype=torch.float64)


In [18]:
import numpy as np
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Generate Z2 x Z2 symmetric dataset
def generate_symmetric_data(n_samples=200):
    x1 = np.random.uniform(-1, 1, n_samples // 4)
    x2 = np.random.uniform(-1, 1, n_samples // 4)

    data = []
    labels = []

    for xi, yi in zip(x1, x2):
        base_point = np.array([xi, yi])
        transformed_points = [
            base_point,
            [-xi, yi],
            [xi, -yi],
            [-xi, -yi]
        ]
        class_label = int(xi * yi > 0)

        for point in transformed_points:
            data.append(point)
            labels.append(class_label)

    return np.array(data), np.array(labels)

# Prepare data
data, labels = generate_symmetric_data()
scaler = StandardScaler()
data = scaler.fit_transform(data)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define quantum device
dev = qml.device("default.qubit", wires=2)

def standard_qnn(weights, x):
    qml.AngleEmbedding(x, wires=[0, 1])
    qml.BasicEntanglerLayers(weights, wires=[0, 1])
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, interface='torch')
def equivariant_qnn(weights, x):
    qml.RY(x[0], wires=0)
    qml.RY(x[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(weights[0], wires=0)
    qml.RY(weights[1], wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0))

# Convert to Torch tensor (Ensure float64 for compatibility with Pennylane)
# Convert to Torch tensor (Ensure float32 for compatibility with PyTorch)
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.float32)

class QuantumModel(nn.Module):
    def __init__(self, quantum_circuit, num_layers=1, num_qubits=2):
        super().__init__()
        # For the equivariant model, the shape should be (num_qubits,)
        # as it has a single layer and expects a 1D tensor for weights.
        if quantum_circuit == equivariant_qnn:  # Check if it's the equivariant model
            self.q_weights = nn.Parameter(0.01 * torch.randn(num_qubits, dtype=torch.float32))  # Shape (num_qubits,)
        else:
            self.q_weights = nn.Parameter(0.01 * torch.randn(num_layers, num_qubits, dtype=torch.float32))  # Shape (num_layers, num_qubits)
        self.q_circuit = quantum_circuit

    def forward(self, x):
        # Apply the quantum circuit to each data point in the batch
        results = [self.q_circuit(self.q_weights, data_point) for data_point in x]
        # Stack the results into a single tensor
        output = torch.stack(results)
        return torch.sigmoid(output).type(torch.float32)

# Train and evaluate both models
def train_model(qnn, X_train, y_train, X_test, y_test):
    optimizer = optim.Adam(qnn.parameters(), lr=0.1)
    loss_fn = nn.BCELoss()

    for epoch in range(100):
        optimizer.zero_grad()
        y_pred = qnn(X_train).squeeze()
        loss = loss_fn(y_pred, y_train)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        y_pred_test = qnn(X_test).squeeze().round()
    acc = accuracy_score(y_test, y_pred_test.numpy())
    return acc

# Train standard QNN
standard_model = QuantumModel(qml.qnode(dev)(standard_qnn), num_layers=1, num_qubits=2)
acc_standard = train_model(standard_model, X_train_torch, y_train_torch, X_test_torch, y_test_torch)
# Train equivariant QNN
equivariant_model = QuantumModel(equivariant_qnn)
acc_equivariant = train_model(equivariant_model, X_train_torch, y_train_torch, X_test_torch, y_test_torch)

print(f"Accuracy of Standard QNN: {acc_standard:.2f}")
print(f"Accuracy of Equivariant QNN: {acc_equivariant:.2f}")


Accuracy of Standard QNN: 0.55
Accuracy of Equivariant QNN: 0.45
